# HuBMAP Release 2
Thomas Hopkins

## Overview

Here is the first attempt at training a UNet for the HuBMAP competition.

The code for this notebook is ported from my GitHub repository here: https://github.com/thomashopkins32/HuBMAP

For loading `pycocotools` in Kaggle, you have to add the dataset in the sidebar on the right.

In [1]:
!cp -r /kaggle/input/pycocotools/ /kaggle/working/pycocotools
!pip install /kaggle/working/pycocotools/pycocotools-2.0.6 --no-index --find-links=/kaggle/working/pycocotools/

cp: cannot stat '/kaggle/input/pycocotools/': No such file or directory
ERROR: Invalid requirement: '/kaggle/working/pycocotools/pycocotools-2.0.6'
Hint: It looks like a path. File '/kaggle/working/pycocotools/pycocotools-2.0.6' does not exist.


In [2]:
import os
import json
import zlib
import base64

from pycocotools import _mask as coco_mask
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import measure
from skimage.draw import polygon
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
from tqdm import tqdm

/home/thomas/anaconda3/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Utilities

Here are some useful utilities that help in debugging, model training, and model evalutation.

I wrote a custom version of mAP for fun so use at your own risk.

In [3]:
def randrange(min_value, max_value, generator=None):
    return (max_value - min_value) * torch.rand(1, generator=generator).item() + min_value


def accuracy(logits, labels):
    preds = torch.argmax(logits, dim=1)
    return torch.count_nonzero(preds == labels) / preds.shape[0]


def memory_usage_stats(model, optimizer, batch_size=1, device='cuda'):
    print(f'Starting memory: {torch.cuda.memory_allocated(device) * 1e-6}')
    model.to(device)
    print(f'After model sent to {device}: {torch.cuda.memory_allocated(device) * 1e-6}')
    for i in range(3):
        sample = torch.randn((batch_size, 3, 512, 512))
        print(f'Step {i}')
        before = torch.cuda.memory_allocated(device) * 1e-6
        out = model(sample.to(device)).sum()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After forward pass: {after}')
        print(f'Memory used by forward pass: {after - before}')
        out.backward()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After backward pass: {after}')
        optimizer.step()
        print(f'After optimizer step: {torch.cuda.memory_allocated(device) * 1e-6}')
    torch.cuda.empty_cache()


def memory_usage_stats_grad_scaler(model, optimizer, batch_size=1, device='cuda'):
    if device != 'cuda':
        print('This function requires device to be "cuda".')
        return
    print(f'Starting memory: {torch.cuda.memory_allocated(device) * 1e-6}')
    model.to(device)
    print(f'After model sent to {device}: {torch.cuda.memory_allocated(device) * 1e-6}')
    scaler = torch.cuda.amp.GradScaler()
    for i in range(3):
        sample = torch.randn((batch_size, 3, 512, 512))
        print(f'Step {i}')
        before = torch.cuda.memory_allocated(device) * 1e-6
        with torch.cuda.amp.autocast(dtype=torch.float16):
            out = model(sample.to(device)).sum()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After forward pass: {after}')
        print(f'Memory used by forward pass: {after - before}')
        scaler.scale(out).backward()
        after = torch.cuda.memory_allocated(device) * 1e-6
        print(f'After backward pass: {after}')
        scaler.step(optimizer)
        print(f'After optimizer step: {torch.cuda.memory_allocated(device) * 1e-6}')
        scaler.update()
    torch.cuda.empty_cache()


def average_precision(prediction, target, iou_threshold=0.6):
    '''
    Computes the average precision (AP) for an instance segmentation task on a single image.

    Parameters
    ----------
    prediction : np.array
        Boolean prediction mask
    target : np.array
        Boolean ground-truth mask
    iou_threshold : float, optional
        Threshold at which to count predictions as positive predictions

    Returns
    -------
    AP : float
        Average precision score
    '''
    pred_regions, pred_region_count = measure.label(prediction, return_num=True)
    target_regions, target_region_count = measure.label(target, return_num=True)
    true_positives = 0
    false_positives = 0
    for p in range(1, pred_region_count + 1):
        pred_region_mask = pred_regions == p
        max_iou = 0.0
        for t in range(1, target_region_count + 1):
            target_region_mask = target_regions == t
            # Compute IoU this region pair
            intersection = np.logical_and(pred_region_mask, target_region_mask)
            union = np.logical_or(pred_region_mask, target_region_mask)
            intersection_count = np.count_nonzero(intersection)
            union_count = np.count_nonzero(union)
            if intersection_count > 0 and union_count > 0:
                iou = intersection_count / union_count
                max_iou = max(max_iou, iou)
        if max_iou > iou_threshold:
            true_positives += 1
        else:
            false_positives += 1
    if (true_positives == 0 and false_positives == 0) or target_region_count == 0:
        return 0.0
    precision = true_positives / (true_positives + false_positives)
    recall = true_positives / target_region_count

    return precision * recall


def mAP(predictions, targets, iou_threshold=0.6):
    averages = 0.0
    predictions = predictions.cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    for p, t in zip(predictions, targets):
        averages += average_precision(p, t, iou_threshold=iou_threshold)
    return averages / len(predictions)


def logits_to_blood_vessel_mask(logits):
    return (torch.argmax(torch.softmax(logits, dim=1), dim=1) == 2).type(torch.long)


def train_one_epoch(epoch, model, train_loader, loss_func, optimizer, writer=None, data_transforms=False, device='cpu', **kwargs):
    total_loss = 0.0
    data_size = len(train_loader)
    model.train()
    for d in train_loader:
        optimizer.zero_grad()
        if data_transforms:
            x = d['transformed_image']
            y = d['transformed_mask']
        else:
            x = d['image']
            y = d['mask']
        x = x.to(device)
        y = y.to(device)
        logits = model(x)
        loss = loss_func(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if writer:
            with torch.no_grad():
                predictions = logits_to_blood_vessel_mask(logits)
                blood_vessel_gt = (y == 2).type(torch.long)
                mAP_value = mAP(predictions, blood_vessel_gt, **kwargs)
                writer.add_scalar("Loss/train", loss.item(), global_step=epoch)
                writer.add_scalar("mAP/train", mAP_value, global_step=epoch)
    
    return total_loss / data_size


def validate_one_epoch(epoch, model, valid_loader, loss_func, writer, data_transforms=False, device='cpu', **kwargs):
    model.eval()
    data_size = len(valid_loader)
    total_loss = 0.0
    total_mAP = 0.0
    with torch.no_grad():
        for i, d in enumerate(valid_loader):
            if data_transforms:
                x = d['transformed_image']
                y = d['transformed_mask']
            else:
                x = d['image']
                y = d['mask']
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = loss_func(logits, y)
            predictions = logits_to_blood_vessel_mask(logits)
            blood_vessel_gt = (y == 2).type(torch.long)
            mAP_value = mAP(predictions, blood_vessel_gt, **kwargs)
            writer.add_scalar("Loss/valid", loss.item(), global_step=epoch)
            writer.add_scalar("mAP/valid", mAP_value, global_step=epoch)
            total_loss += loss.item()
            total_mAP += mAP_value

    return total_loss / data_size, total_mAP / data_size
    


def oid_mask_encoding(mask):
    '''
    Taken from https://gist.github.com/pculliton/209398a2a52867580c6103e25e55d93c
    using the competition page
    '''
    # check input mask --
    if mask.dtype != bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str


def kaggle_prediction(image_id, logits):
    '''
    Encodes the raw output of the model into the submission format
    
    Parameters
    ----------
    image_id : str
        Identifier of the predicted image
    logits : torch.tensor
        Raw output of the model with shape (3, 512, 512)
    
    Returns
    -------
    dict
        Submission entry with the headers (id, height, width, prediction_string)
    '''

    # keep raw prediction probabilities
    preds = torch.softmax(logits, dim=0)

    # convert blood vessel predictions to the mask
    prediction = logits_to_blood_vessel_mask(logits.unsqueeze(0)).squeeze().detach().cpu().numpy()

    # label predicted connected components
    pred_regions, pred_region_count = measure.label(prediction, return_num=True)

    prediction_string = ''
    for p in range(1, pred_region_count + 1):
        # get the coordinates of the current region
        x, y = np.where(pred_regions == p)

        # separate the binary mask
        mask = np.zeros_like(pred_regions, dtype=bool)
        mask[(x, y)] = True

        # encode the separated mask
        encoding = oid_mask_encoding(mask)

        # average the prediction probabilities of the current region
        region_prob = torch.mean(preds[2, x, y])

        prediction_string += f'0 {region_prob.item()} {encoding.decode("utf-8")} '
    
    return {
        'id': image_id,
        'height': 512,
        'width': 512,
        'prediction_string': prediction_string.strip()
    }


def save_model_checkpoint(path, epoch, model, optimizer, loss, scheduler=None):
    checkpoint_dict = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'training_loss': loss,
    }
    if scheduler:
        checkpoint_dict['scheduler_state_dict'] = scheduler.state_dict()
    else:
        checkpoint_dict['scheduler_state_dict'] = None
    torch.save(checkpoint_dict, path)


def load_model_checkpoint(path, model, optimizer=None, scheduler=None):
    checkpoint = torch.load(path)
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['model_state_dict'])
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler and checkpoint['scheduler_state_dict']:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    return epoch

## Architecture

Here is the UNet architecture implementation.

See the UNet paper for more information: https://arxiv.org/pdf/1505.04597v1.pdf

In [4]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        '''
        Note: We pad the convolutions here since we know the input image size is always 512x512.
        Otherwise we would do the "Overlap-tile strategy" presented in the u-net paper.
        '''
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, xx):
        return self.model(xx)


class UNet2d(nn.Module):
    ''' Paper: https://arxiv.org/pdf/1505.04597v1.pdf '''
    def __init__(self):
        super().__init__()
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()

        # Down
        self.conv1 = ConvBlock(3, 64)
        self.conv2 = ConvBlock(64, 128)
        self.conv3 = ConvBlock(128, 256)
        self.conv4 = ConvBlock(256, 512)
        self.conv5 = ConvBlock(512, 1024)

        # Up
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.conv6 = ConvBlock(1024, 512)
        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.conv7 = ConvBlock(512, 256)
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv8 = ConvBlock(256, 128)
        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv9 = ConvBlock(128, 64)
        self.conv_out = nn.Conv2d(64, 3, kernel_size=1)

    def forward(self, xx):
        # Down
        x1 = self.conv1(xx)
        xx = self.max_pool(x1)
        x2 = self.conv2(xx)
        xx = self.max_pool(x2)
        x3 = self.conv3(xx)
        xx = self.max_pool(x3)
        x4 = self.conv4(xx)
        xx = self.max_pool(x4)
        xx = self.conv5(xx)

        # Up
        xx = self.upconv1(xx)
        xx = torch.cat((x4, xx), dim=1)
        xx = self.conv6(xx)
        xx = self.upconv2(xx)
        xx = torch.cat((x3, xx), dim=1)
        xx = self.conv7(xx)
        xx = self.upconv3(xx)
        xx = torch.cat((x2, xx), dim=1)
        xx = self.conv8(xx)
        xx = self.upconv4(xx)
        xx = torch.cat((x1, xx), dim=1)
        xx = self.conv9(xx)
        return self.conv_out(xx)

## Dataset

Here is the dataset that loads the images, annotations, and peforms data augmentation.

We allow for the prediction of the background (label 0), glomerulus (label 1), and blood vessels (label 2).

In [5]:

class HuBMAP(Dataset):
    ''' Training dataset for the HuBMAP Kaggle Competition '''
    def __init__(self, data_dir=os.path.join('.', 'data'), submission=False, include_unsure=False):
        self.data_dir = data_dir
        self.submission = submission
        self.generator = torch.Generator()
        self.generator.manual_seed(16)
        self.image_ids = []
        self.images = []
        self.masks = [] # target structure

        if not submission:
            # Load in the training labels
            with open(os.path.join(data_dir, 'polygons.jsonl'), 'r') as polygons_file:
                polygons = list(polygons_file)
            self.polygons = [json.loads(p) for p in polygons]

            # Load all of the training images and annotations into memory
            self.img_size = 512
            print("Loading in images and converting annotations to polygon masks...")
            for poly in tqdm(self.polygons):
                id = poly['id']
                # Get image using id
                image = Image.open(os.path.join(data_dir, 'train', f'{id}.tif'))
                self.image_ids.append(id)
                self.images.append(image)

                # Get all of the different annotations for this image
                ## A single image can have multiple annotations for each type
                blood_vessel_coords = []
                glomerulus_coords = []
                unsure_coords = []
                ## Load in the type and coordinates for each annotation
                annotations = poly['annotations']
                for ann in annotations:
                    type = ann['type']
                    assert len(ann['coordinates']) <= 1
                    coordinates = ann['coordinates'][0]
                    row_indices = [c[0] for c in coordinates] 
                    col_indices = [c[1] for c in coordinates]
                    row_indices, col_indices = polygon(row_indices, col_indices, (self.img_size, self.img_size))
                    coordinates = list(zip(row_indices, col_indices))
                    if type == 'blood_vessel':
                        blood_vessel_coords.append(coordinates)
                    elif type == 'glomerulus':
                        glomerulus_coords.append(coordinates)
                    else:
                        unsure_coords.append(coordinates) 
                mask = self.coordinates_to_mask([item for sublist in glomerulus_coords for item in sublist])
                if include_unsure and unsure_coords is not None and len(unsure_coords) > 0:
                    uns_coords = torch.tensor([item for sublist in unsure_coords for item in sublist])
                    mask[uns_coords[:, 1], uns_coords[:, 0]] = 2
                if blood_vessel_coords is not None and len(blood_vessel_coords) > 0:
                    bv_coords = torch.tensor([item for sublist in blood_vessel_coords for item in sublist])
                    # coordinates are (x, y) like a grid
                    mask[bv_coords[:, 1], bv_coords[:, 0]] = 2
                self.masks.append(mask)
            print("Done.")
        else:
            for image_file in os.listdir(os.path.join(data_dir, 'test')):
                image = Image.open(os.path.join(data_dir, 'test', image_file))
                id = image_file.split('.')[0]
                self.image_ids.append(id)
                self.images.append(image)
                self.masks.append(torch.zeros((512, 512)))

        self.test_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((512, 512), antialias=True)
        ])

    def transform(self, image, mask):
        image = F.to_tensor(image)
        resize = transforms.Resize(size=(512, 512), antialias=True)
        image = resize(image)
        mask = mask.unsqueeze(0)

        # horizontal flip
        if torch.rand(1, generator=self.generator).item() > 0.5:
            image = F.hflip(image)
            mask = F.hflip(mask)

        # vertical flip
        if torch.rand(1, generator=self.generator).item() > 0.5:
            image = F.vflip(image)
            mask = F.vflip(mask)
        
        # random affine (fill in with white pixels)
        # see https://github.com/thomashopkins32/HuBMAP/issues/6#issuecomment-1656778016
        angle = 0 # no rotation (weird borders)
        shear = 0 # no shear (weird borders)
        horizontal_translation_factor = randrange(0.0, 0.03, generator=self.generator)
        ht_min = ceil(-512 * horizontal_translation_factor)
        ht_max = floor(512 * horizontal_translation_factor)
        vertical_translation_factor = randrange(0.0, 0.03, generator=self.generator)
        vt_min = ceil(-512 * vertical_translation_factor)
        vt_max = floor(-512 * vertical_translation_factor)
        if ht_min < ht_max:
            horizontal_translation = torch.randint(
                ht_min,
                ht_max,
                (1,),
                generator=self.generator).item()
        else:
            horizontal_translation = 0
        if vt_min < vt_max:
            vertical_translation = torch.randint(
                vt_min,
                vt_max,
                (1,),
                generator=self.generator).item()
        else:
            vertical_translation = 0
        translation = (horizontal_translation, vertical_translation)
        scale = randrange(1.0, 1.1, generator=self.generator)
        fill = 1.0
        image = F.affine(image, angle, translation, scale, shear, fill=fill)
        mask = F.affine(mask, angle, translation, scale, shear, fill=0)

        # brightness
        brightness_factor = randrange(0.8, 1.2, generator=self.generator)
        image = F.adjust_brightness(image, brightness_factor)

        # contrast
        contrast_factor = randrange(0.8, 1.2, generator=self.generator)
        image = F.adjust_contrast(image, contrast_factor)

        # saturation
        saturation_factor = randrange(0.8, 1.2, generator=self.generator)
        image = F.adjust_saturation(image, saturation_factor)

        return image, mask.squeeze()

    def coordinates_to_mask(self, coordinates):
        if coordinates is None or coordinates == []:
            return torch.zeros((self.img_size, self.img_size), dtype=torch.long)
        coords = torch.tensor(coordinates)
        mask = torch.zeros((self.img_size, self.img_size), dtype=torch.bool)
        # coordinates are (x, y) like a grid
        mask[coords[:, 1], coords[:, 0]] = True
        return mask.type(torch.long)

    def __getitem__(self, i):
        if self.submission:
            image = self.test_transforms(self.images[i])
            mask = self.masks[i]
            transformed_image = image
            transformed_mask = mask
        else:
            transformed_image, transformed_mask = self.transform(self.images[i], self.masks[i])
            image = self.test_transforms(self.images[i])
            mask = self.masks[i]
        return {
            'id': self.image_ids[i],
            'image': image,
            'mask': mask,
            'transformed_image': transformed_image,
            'transformed_mask': transformed_mask.type(torch.long),
        }

    def __len__(self):
        return len(self.images)

## CUDA Memory Usage

It's important to see how much memory our model is using during training. Since this is a fairly large model with a large input (512x512 images), we need to know how large a batch size we can use on a single device. This will make training easier since our batch normalization will have better statistics the larger the batch size we use.


Using a very low batch size could actually decrease performance when using batch normalization since it's statistics will be very noisy.

In [6]:
#model = UNet2d()
#optimizer = optim.Adam(model.parameters())
#memory_usage_stats(model, optimizer, batch_size=9, device='cuda')

## Training

Here is where we actually train the UNet.

In [7]:
os.mkdir('checkpoints')
# PARAMETERS
RUN_NAME = 'release2_run1'
BATCH_SIZE = 9
LR = 3e-4
WD = 0.0
MOMENTUM = 0.0
DATA_TRANSFORMATIONS = True
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
VALID_STEP = 5
RNG = 32
EPOCH_START = 0
EPOCH_END = 200
CHECKPOINT_STEP = 5
CHECKPOINT_LOAD_PATH = None
CHECKPOINT_SAVE_PATH = os.path.join('checkpoints', f'{RUN_NAME}.pt')
INCLUDE_UNSURE = True

torch.manual_seed(RNG)

writer = SummaryWriter()
dataset = HuBMAP(include_unsure=INCLUDE_UNSURE)
generator = torch.Generator().manual_seed(RNG)
train_data, valid_data = random_split(dataset, [0.9, 0.1], generator=generator)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory=False)
valid_loader = DataLoader(valid_data, batch_size=1, shuffle=False, pin_memory=False)
model = UNet2d().to(DEVICE)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max')
if CHECKPOINT_LOAD_PATH:
    EPOCH_START = load_model_checkpoint(CHECKPOINT_LOAD_PATH, model, optimizer, scheduler=scheduler) 

for e in tqdm(range(EPOCH_START + 1, EPOCH_END + 1)):
    if e % VALID_STEP == 0:
        loss = train_one_epoch(
            e,
            model,
            train_loader,
            loss_func,
            optimizer,
            writer=writer,
            data_transforms=DATA_TRANSFORMATIONS,
            device=DEVICE
        )
        val_loss, val_metric = validate_one_epoch(
            e,
            model,
            valid_loader,
            loss_func,
            writer,
            data_transforms=False,
            device=DEVICE
        )
        if scheduler:
            scheduler.step(val_metric)
    else:
        loss = train_one_epoch(
            e,
            model,
            train_loader,
            loss_func,
            optimizer, 
            data_transforms=DATA_TRANSFORMATIONS,
            device=DEVICE
        )
    if e % CHECKPOINT_STEP == 0:
        save_model_checkpoint(CHECKPOINT_SAVE_PATH, e, model, optimizer, loss, scheduler=scheduler)
    writer.add_scalar('gpu_memory_usage', torch.cuda.memory_allocated(DEVICE), global_step=e)
writer.close()

FileNotFoundError: [Errno 2] No such file or directory: './data/polygons.jsonl'

## Submission
Here is where we predict on the test images for submission.

In [ ]:
# PARAMETERS
RUN_NAME = 'release2_run1'
BATCH_SIZE = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
RNG = 32
CHECKPOINT_LOAD_PATH = os.path.join('checkpoints', f'{RUN_NAME}.pt')

torch.manual_seed(RNG)

dataset = HuBMAP(data_dir=os.path.join('/kaggle', 'input', 'hubmap-hacking-the-human-vasculature'), submission=True)
#generator = torch.Generator().manual_seed(RNG)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=False)
#model = UNet2d().to(DEVICE)
#if CHECKPOINT_LOAD_PATH:
    #EPOCH_START = load_model_checkpoint(CHECKPOINT_LOAD_PATH, model)
model.eval()

submission_entries = pd.DataFrame(columns=['id', 'height', 'width', 'prediction_string'])
for d in loader:
    id = d['id']
    x = d['image']
    x = x.to(DEVICE)
    logits = model(x)
    for i in range(logits.shape[0]):
        submission = kaggle_prediction(id, logits[i, :, :, :])
        submission_df = pd.DataFrame(submission)
        submission_entries = pd.concat((submission_entries, submission_df), axis=0)

submission_entries.to_csv('submission.csv', index=False)